## Build and Test API with dummy model

In [ ]:
from electricity_price_predictor.data import get_all, get_weather
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import joblib

Load data

In [ ]:
df = get_all()

### Features and target defined

In [ ]:
train = df.iloc[:-1]
test = df.iloc[-1:]

In [ ]:
X_train = train.drop('price', axis=1)
y_train = train.price.values

X_test = test.drop('price', axis=1)
y_test = test.price.values

### Model

In [ ]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('estimator', LinearRegression())
])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.predict(X_test)

#### Save model

Model saved

In [ ]:
#model_name = '../test_model.joblib'
#joblib.dump(pipe, model_name)

Test set saved

In [ ]:
#dummy_test = '../test_testset.joblib'
#joblib.dump(X_test, dummy_test)

## App api

In [ ]:
from flask import Flask, escape, request
app = Flask(__name__)

In [ ]:
@app.route('/')
def hello():
    # get param from http://127.0.0.1:5000/?name=value
    name = request.args.get("name", "World")
    return f'Hello, {escape(name)}!'

In [ ]:
@app.route('/predict_day_ahead')
def day_head():
    test = joblib.load('../test_testset.joblib')

    model = joblib.load('../test_model.joblib')
    pred = model.predict(test)[0]
    
    return {'test_values': test, 'day-ahead prediction': pred}

In [ ]:
X_test

In [ ]:
import streamlit as st

In [ ]:
import numpy as np
import pandas as pd

st.markdown("""# This is a header
## This is a sub header
This is text""")

df = pd.DataFrame({
          'first column': list(range(1, 11)),
          'second column': np.arange(10, 101, 10)
        })

# this slider allows the user to select a number of lines
# to display in the dataframe
# the selected value is returned by st.slider
line_count = st.slider('Select a line count', 1, 10, 3)

# and used in order to select the displayed lines
head_df = df.head(line_count)

head_df

In [ ]:
from electricity_price_predictor.data import get_weather

In [ ]:
def get_weather(path='../raw_data/weather_2015_2020.csv'):
    df = pd.read_csv(path)

    df['dt'] = pd.to_datetime(df.dt)

    # drop unnecessary columns
    to_drop = ['dt_iso','timezone','lat', 'lon','sea_level','grnd_level',
               'rain_1h','rain_3h', 'pressure', 'snow_1h', 'snow_3h',
               'temp_min','temp_max','weather_id', 'weather_description',
               'weather_icon', 'wind_deg']
    df = df.drop(to_drop, axis=1)

    # population of each city in the df
    pop = {'Aarhus': 349_983,
        'Odense': 204_895,
        'Aalborg': 217_075,
        'Esbjerg': 115_748,
        'Vejle': 111_743,
        'Randers': 96_559,
        'Viborg': 93_819,
        'Kolding': 89_412,
        'Silkeborg': 89_328,
        'Herning': 86_348,
        'Horsens': 83_598}

    df['population'] = [pop[city] for city in df.city_name]

    # numeric weather values as affects demand or supply
    numeric_cols = ['temp', 'feels_like', 'humidity',  'clouds_all','wind_speed']

    weather_df = pd.DataFrame()

    #for the numeric columns, group by datetime and average according to their population weight
    for col in numeric_cols:
    #group by the datecolumn for each element in the column average it by it's weight
        weather_df[col] = df.groupby(df.dt).apply(lambda x : np.average(x[col], weights=x.population))

    # check for missing indices
    missing_idx = pd.date_range(start = '2015-01-01', end = '2020-11-24', freq='H' ).difference(weather_df.index)

    # impute missing indices with average of bounding rows
    for idx in missing_idx:
        weather_df.loc[idx] = weather_df.loc[pd.to_datetime(idx) - timedelta(hours= 1)] + \
                      weather_df.loc[pd.to_datetime(idx) + timedelta(hours= 1)] / 2

    weather_df = weather_df.sort_index()

    return weather_df

In [ ]:
weather = get_weather()

In [ ]:
weather

In [ ]:
!pwd

In [ ]:
from electricity_price_predictor.data import get_all

In [ ]:
df = get_all()

In [ ]:
df.head()

In [ ]:
X = df.price.values

In [ ]:
y = df.wind_prod.values

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(y, X)

In [ ]:
df = get_weather()

In [ ]:
df

## Testing API for new prices generation

In [1]:
import requests
import json
import xmltodict
import pandas as pd
from datetime import timezone, datetime, timedelta
from electricity_price_predictor.data import get_new_price

In [2]:
start = datetime(2020, 12, 3, 0, 0)
stop = start + timedelta(days=1)
key= " "

In [5]:
get_new_price(start, stop)

,price
2020-12-03 00:00,9.23
2020-12-03 01:00,8.48
2020-12-03 02:00,8.19
2020-12-03 03:00,8.35
2020-12-03 04:00,8.94
2020-12-03 05:00,12.90
2020-12-03 06:00,17.90
2020-12-03 07:00,46.71
2020-12-03 08:00,61.07
2020-12-03 09:00,54.32
